In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [11]:
#reading the data
df=pd.read_csv('data/data.csv')
df['summary'][0]


' Drowned Wednesday is the first Trustee among the Morrow Days who is on Arthur\'s side and wishes the Will to be fulfilled. She appears as a leviathan/whale and suffers from Gluttony. The book begins when Leaf is visiting Arthur and they are discussing the invitation that Drowned Wednesday sent him. Arthur had been admitted to hospital because of the damage done to his leg when he attempted to enter Tuesday\'s Treasure Tower. Suddenly, the hospital room becomes flooded with water as the two are transported to the Border Sea of the House. Leaf is snatched away by a large ship with green sails, known as the Flying Mantis, while Arthur remains in his bed. When the Medallion given him by the immortal called the Mariner apparently fails to summon help, Arthur is without hope. Eventually, a buoy marking the pirate Elishar Feverfew\'s treasure floats toward him. As soon as Arthur opens it, his hand is marked with a bloody red colour. Arthur now has the Red Hand, by which Feverfew marks whoev

In [9]:
df.genre.value_counts()

genre
Fantasy             500
Science Fiction     500
Crime Fiction       500
Historical novel    500
Horror              500
Thriller            500
Name: count, dtype: int64

In [6]:
df.head()

,Unnamed: 0,book_id,book_name,genre,summary
0,0,3248537,Drowned Wednesday,Fantasy,Drowned Wednesday is the first Trustee among ...
1,1,27796919,The Lost Hero,Fantasy,"As the book opens, Jason awakens on a school ..."
2,2,3910776,The Eyes of the Overworld,Fantasy,Cugel is easily persuaded by the merchant Fia...
3,3,5969644,Magic's Promise,Fantasy,The book opens with Herald-Mage Vanyel return...
4,4,3173445,Taran Wanderer,Fantasy,Taran and Gurgi have returned to Caer Dallben...


In [7]:
#dropping unnecessary columns
df.drop("Unnamed: 0",axis='columns',inplace=True)
df.drop('book_id',axis='columns',inplace=True)

In [8]:
df.head()

,book_name,genre,summary
0,Drowned Wednesday,Fantasy,Drowned Wednesday is the first Trustee among ...
1,The Lost Hero,Fantasy,"As the book opens, Jason awakens on a school ..."
2,The Eyes of the Overworld,Fantasy,Cugel is easily persuaded by the merchant Fia...
3,Magic's Promise,Fantasy,The book opens with Herald-Mage Vanyel return...
4,Taran Wanderer,Fantasy,Taran and Gurgi have returned to Caer Dallben...


In [9]:
df.isnull().sum()

book_name    0
genre        0
summary      0
dtype: int64

In [10]:
df.shape

(3000, 3)

In [11]:
df['book_name'].nunique()

2970

In [12]:
#do some feature  scaling

In [13]:
df.duplicated().sum()

0

In [14]:
df['tag']=df['book_name'].str.split()+df['summary'].str.split()

In [15]:
df['tag']=df['tag'].apply(lambda x:' '.join(x))

In [16]:
#removing book_name and summary becuase we don't need it further more
df.drop('summary',axis='columns',inplace=True)
df.drop('book_name',axis='columns',inplace=True)

In [17]:
df.head()

,genre,tag
0,Fantasy,Drowned Wednesday Drowned Wednesday is the fir...
1,Fantasy,"The Lost Hero As the book opens, Jason awakens..."
2,Fantasy,The Eyes of the Overworld Cugel is easily pers...
3,Fantasy,Magic's Promise The book opens with Herald-Mag...
4,Fantasy,Taran Wanderer Taran and Gurgi have returned t...


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
w=WordNetLemmatizer()

In [20]:
df['tag']=df['tag'].apply(lambda x: ' '.join([w.lemmatize(i) for i in nltk.word_tokenize(x) if i not in stopwords.words('english')]))

In [21]:
X_train,X_test,y_train,y_test=train_test_split(df.tag,df.genre,test_size=0.25,random_state=35)

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
lb=LabelEncoder()
tf=TfidfVectorizer(max_features=150000)


In [24]:
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

In [25]:
X_train=tf.fit_transform(X_train)
X_test=tf.transform(X_test)

In [26]:
rfc=RandomForestClassifier()

In [27]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [28]:
ypre=rfc.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(ypre,y_test)

0.6386666666666667

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
lr=LogisticRegression()

In [32]:
lr.fit(X_train,y_train)

LogisticRegression()

In [33]:
ypre=lr.predict(X_test)

In [34]:
accuracy_score(ypre,y_test)

0.7266666666666667

In [68]:
from sklearn.model_selection import GridSearchCV

In [64]:
params={
    'C':[100,200,300,400]
}


In [65]:
gr=GridSearchCV(lr,params,cv=5,scoring='accuracy')

In [66]:
gr.fit(X_train,y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [100, 200, 300, 400]}, scoring='accuracy')

In [67]:
gr.best_score_

0.7151111111111111

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Define parameters for each algorithm
param_grid = {
    'RandomForest': {'n_estimators': [10, 50, 100], 'max_depth': [None, 5, 10]},
    'SVM': {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 'scale']},
    'KNN': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
    'LogisticRegression': {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']},
    'DecisionTree': {'max_depth': [None, 5, 10], 'min_samples_split': [2, 5, 10]},
    'NaiveBayes': {},
    'MLP': {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'alpha': [0.0001, 0.001, 0.01]}
}

# Instantiate classifiers
classifiers = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTree': DecisionTreeClassifier(),
    'NaiveBayes': GaussianNB(),
    'MLP': MLPClassifier()
}

# Perform GridSearchCV for each classifier
for clf_name, clf in classifiers.items():
    if clf_name in param_grid:
        params = param_grid[clf_name]
        grid_search = GridSearchCV(clf, params, cv=5)
        grid_search.fit(X_train, y_train)  # Assuming X_train and y_train are your training data
        print(f"Best parameters for {clf_name}: {grid_search.best_params_}")


Best parameters for RandomForest: {'max_depth': None, 'n_estimators': 100}
Best parameters for SVM: {'C': 10, 'gamma': 0.1}
Best parameters for KNN: {'n_neighbors': 7, 'weights': 'distance'}


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Best parameters for LogisticRegression: {'C': 10, 'penalty': 'l2'}
Best parameters for DecisionTree: {'max_depth': None, 'min_samples_split': 10}


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\naive_bayes.py", line 263, in fit
    return self._partial_fit(
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\naive_bayes.py", line 423, in _partial_fit
    X, y = self._validate_data(X,

In [71]:
'''Best parameters for RandomForest: {'max_depth': None, 'n_estimators': 100}
Best parameters for SVM: {'C': 10, 'gamma': 0.1}
Best parameters for KNN: {'n_neighbors': 3, 'weights': 'distance'}
Best parameters for LogisticRegression: {'C': 10, 'penalty': 'l2'}
Best parameters for DecisionTree: {'max_depth': None, 'min_samples_split': 5}
'''

"Best parameters for RandomForest: {'max_depth': None, 'n_estimators': 100}\nBest parameters for SVM: {'C': 10, 'gamma': 0.1}\nBest parameters for KNN: {'n_neighbors': 3, 'weights': 'distance'}\nBest parameters for LogisticRegression: {'C': 10, 'penalty': 'l2'}\nBest parameters for DecisionTree: {'max_depth': None, 'min_samples_split': 5}\n"

In [74]:
classifiers = {
    'RandomForest': RandomForestClassifier(max_depth=None,n_estimators=100),
    'SVM': SVC(C=10,gamma=0.1),
    'KNN': KNeighborsClassifier(n_neighbors=3,weights='distance'),
    'LogisticRegression': LogisticRegression(C=100,penalty='l2'),
    'DecisionTree': DecisionTreeClassifier(max_depth=None,min_samples_split=5)
}

In [80]:
for clf_name,model in classifiers.items():
    model=model
    model.fit(X_train,y_train)
    ypr=model.predict(X_test)
    print('score:',accuracy_score(ypre,y_test))

score: 0.7053333333333334
score: 0.7053333333333334
score: 0.7053333333333334


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


score: 0.7053333333333334
score: 0.7053333333333334


In [51]:
model=LogisticRegression(C=10,penalty='l2')

In [52]:
model.fit(X_train,y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10)

In [53]:
ypr=model.predict(X_test)
accuracy_score(ypr,y_test)

0.7226666666666667

In [54]:
import pickle
pickle.dump(model,open('model.pkl','wb'))

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer

def lemmi_stopwords(text):
    text = text.lower()
    lm = WordNetLemmatizer()
    tokenize_words = nltk.word_tokenize(text)
    lemmitize_text = [lm.lemmatize(word) for word in tokenize_words]
    return ' '.join(lemmitize_text)


In [6]:
lemmi_stopwords('My name is Ayaz')

'my name is ayaz'